In [ ]:
from langchain import PromptTemplate

## Partialing a prompt template and injecting values

We can pass `key=value` keyword arguments to fill (partially if desired) the prompt to refine the template,
until the final `format` call.

We can also pass a function that dynamically provides the values to fill the template (see below)

In [ ]:
template = """You are a support assistant users of an accounting software product called MagicContability.
Your answers should be adapted to the description of the user given below.
In particular, ONLY IF the user is not a paying user you can try to suggest,
when appropriate, to switch to the paid plan.

- the user's favorite programming language is {user_language}
- does the user have a paid plan? {paid_plan}
- does the user prefer an informal communication style? {informal_style}

Here is the user's question: "{user_question}"
YOUR ANSWER:
"""

In [ ]:
user_data = {
    'user_language': 'Javascript',
    'paid_plan': 'NO',
    'informal_style': 'YES',
}
user_question = "Is P equal to NP?"

In [ ]:
prompt1 = PromptTemplate(
    template=template,
    input_variables=["user_language", "paid_plan", "informal_style", "user_question"],
)
# Here the idea is that three of the above would come from DB lookup once user ID is known, the last one is 'live'
partial_prompt1 = prompt1.partial(**user_data)

In [ ]:
print(partial_prompt1.format(user_question=user_question))

#### Alternative way to partial

In [ ]:
prompt2 = PromptTemplate(
    template=template,
    input_variables=["user_question"],
    partial_variables=user_data,
)
print(prompt2.format(user_question=user_question))

#### Partialing with functions

The limitations is that it's not clear how to handle dependency injection (e.g. a DB connection or even more a user ID for lookup) in the function

In [ ]:
prompt3 = PromptTemplate(
    template=template,
    input_variables=["user_language", "paid_plan", "informal_style", "user_question"],
)

# 'getters'
def _get_user_language(): return 'Python'
def _get_paid_plan(): return 'YES'
def _get_informal_style(): return 'NO'

partial_prompt3 = prompt3.partial(
    user_language=_get_user_language,
    paid_plan=_get_paid_plan,
    informal_style=_get_informal_style
)

In [ ]:
print(partial_prompt3.format(user_question=user_question))

#### Or inline partialing

In [ ]:
# 'getters'
def _get_user_language(): return 'Erlang'
def _get_paid_plan(): return 'NO'
def _get_informal_style(): return 'NO'

prompt4 = PromptTemplate(
    template=template,
    input_variables=["user_question"],
    partial_variables={
        'user_language': _get_user_language,
        'paid_plan': _get_paid_plan,
        'informal_style': _get_informal_style,
    },
)

In [ ]:
print(prompt4.format(user_question=user_question))

## Dependencies and/or richer parameters to "getters"

Little experiments to see if a DB-related partialing is somehow possible:
- injecting a DB dependency into the "getters"
- ability to pass parameters to the getters (e.g. "user_id") to pick the values for the prompt
- interaction with partialing

#### With the prompt templates

In [ ]:
dtemplate = """You are helping a user.
The user has name "{user_name}" and support level "{support_level}".

Try to answer his question.
USER'S QUESTION: {user_question}
YOUR ANSWER:
"""

In [ ]:
# 'getters'
def _get_user_name(user_id): return 'John %s' % user_id
def _get_support_level(user_id): return 'SUPPT_%s' % user_id

dprompt1 = PromptTemplate(
    template=dtemplate,
    input_variables=["user_question"],
    partial_variables={
        'user_name': _get_user_name,
        'support_level': _get_support_level,
    },
)

This is impossible, as can be seen by the `v()` [here](https://github.com/hwchase17/langchain/blob/206c87d5255687856401c94bcc2c324c5daacac4/langchain/prompts/base.py#L159)

In [ ]:
print(dprompt1.format(user_question='What is life?', user_id='usr001'))

Same route to explicitly call `partial()`

In [ ]:
dprompt2 = PromptTemplate(
    template=dtemplate,
    input_variables=["user_question", "support_level", "user_name"],
)

# 'getters'
def _get_user_name(user_id): return 'John %s' % user_id
def _get_support_level(user_id): return 'SUPPT_%s' % user_id

partial_dprompt2 = dprompt2.partial(
    user_name=_get_user_name,
    support_level=_get_support_level,
)

print(partial_dprompt2.format(user_question='What is life?', user_id='usr001'))

## Creating a custom prompt template

This seems to be the way. [Reference](https://python.langchain.com/en/latest/modules/prompts/prompt_templates/examples/custom_prompt_template.html#).

In [ ]:
# from cassandra.cluster import Session

In [ ]:
from DependencyfulPromptTemplate import DependencyfulPromptTemplate

In [ ]:
def _dep_get_user_name(db, user_id): return 'John %s (from %s)' % (user_id, db)
def _dep_get_support_level(db, user_id): return 'SUPPT_%s (from %s)' % (user_id, db)

def myGetter(deps, user_id):
    db = deps['db']
    return {
        'user_name': _dep_get_user_name(db, user_id),
        'support_level': _dep_get_support_level(db, user_id),
    }

promptx = DependencyfulPromptTemplate(
    template=dtemplate,
    dependencies={'db': 'myDatabase'},
    getter=myGetter,
    input_variables=["user_question", "support_level", "user_name"],
)

resultx = promptx.format(
    user_id='USR01',
    user_question='What is love?',
)
print(resultx)

#### Partialing of the above

Since all invocations are deferred to the end anyway, it is possible to leverage the general `partial` mechanism by langchain.

In [ ]:
partial_promptx1 = promptx.partial(user_id='00aaa')
print(partial_promptx1.format(
    user_question='Quid?',
))

In [ ]:
partial_promptx1 = promptx.partial(user_question='What do I do?')
print(partial_promptx1.format(
    user_id='u444',
))

In [ ]:
partial_promptx1 = promptx.partial(
    user_id='baba000',
    user_question='How come?',
)
print(partial_promptx1.format())

## Specializing the custom prompt template class into a CassandraDependencyTemplate

- dependencies are the DB Session and the keyspace
- placeholder variables in templates can refer to a mapping from their name to either (a) (table, column) or (b) (table, function(row))
- several tables can be queried if necessary, with schema inspection to properly restrict the where clause. They are supposed to return at least one row, the first is chosen if more than one

In [ ]:
from cassandraPromptTemplate import createCassandraPromptTemplate

In [ ]:
from cqlsession import getCqlSession
astraSession = getCqlSession()

In [ ]:
ctemplate0 = """Please answer a question from a user.
Keep in mind that the user's age is {user_age} and they live in a city with
nickname {city_nickname}.

USER'S QUESTION: {user_question}
YOUR ANSWER:
"""

In [ ]:
cassPrompt = createCassandraPromptTemplate(
    session=astraSession,
    keyspace='pqdemo',
    template=ctemplate0,
    input_variables=['city', 'name', 'user_question'],
    field_mapper={
        'user_age': ('people', 'age'),
        'city_nickname': ('nickname_by_city', 'nickname'),
    },
)

In [ ]:
print(cassPrompt.format(city='turin', name='beppe', user_question='Is functional programming fun?'))

In [ ]:
cassPartialPrompt = cassPrompt.partial(city='lisbon', name='Pedro')

In [ ]:
print(cassPartialPrompt.format(user_question='Em verdade, o que quiseres?'))

### Pretty cool, eh?

Alternatively, functions of a `Row` object can be passed in place of column names:

In [ ]:
cassPromptF = createCassandraPromptTemplate(
    session=astraSession,
    keyspace='pqdemo',
    template=ctemplate0,
    input_variables=['city', 'name', 'user_question'],
    field_mapper={
        'user_age': ('people', lambda row: row.age * 1000),
        'city_nickname': ('nickname_by_city', lambda row: '%s (%s)' % (row.nickname.upper(), row.city)),
    },
)

In [ ]:
print(cassPromptF.format(city='liverpool', name='Vivian', user_question='Is functional programming fun?'))